In [1]:
import os
import json

# Replace these with your actual credentials
kaggle_username = "diaaalmohamad"
kaggle_key = "609a7a743532bebd2dee77c5330d5a30"

# Create the .kaggle directory if it doesn't exist
os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)

# Write kaggle.json
kaggle_config = {"username": kaggle_username, "key": kaggle_key}
with open(os.path.expanduser("~/.kaggle/kaggle.json"), "w") as f:
    json.dump(kaggle_config, f)

# Fix permissions (safe to skip on Windows)
os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"), 0o600)


In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile, os

def get_kaggle_competition(competition, path="data", unzip=True):
    api = KaggleApi()
    api.authenticate()
    os.makedirs(path, exist_ok=True)
    
    print(f"Downloading '{competition}'...")
    api.competition_download_files(competition, path=path)
    
    zip_path = os.path.join(path, f"{competition}.zip")
    if unzip:
        print("Extracting files...")
        with zipfile.ZipFile(zip_path, "r") as z:
            z.extractall(path)
        os.remove(zip_path)
        print("✅ Done!")
    else:
        print(f"Downloaded to {zip_path}")

# Example use:
get_kaggle_competition("us-patent-phrase-to-phrase-matching")


Extracting files...
✅ Done!


Get training data

In [1]:
import pandas as pd
import numpy as np

train_df = pd.read_csv("data/train.csv")
print(train_df.head())

                 id     anchor                  target context  score
0  37d61fd2272659b1  abatement  abatement of pollution     A47   0.50
1  7b9652b17b68b7a4  abatement          act of abating     A47   0.75
2  36d72442aefd8232  abatement         active catalyst     A47   0.25
3  5296b0c19e1ce60e  abatement     eliminating process     A47   0.50
4  54c1e3b9184cb5b6  abatement           forest region     A47   0.00


In [2]:
train_df.describe(include='all')

,id,anchor,target,context,score
count,36473,36473,36473,36473,36473.000000
unique,36473,733,29340,106,NaN
top,37d61fd2272659b1,component composite coating,composition,H01,NaN
freq,1,152,24,2186,NaN
mean,NaN,NaN,NaN,NaN,0.362062
std,NaN,NaN,NaN,NaN,0.258335
min,NaN,NaN,NaN,NaN,0.000000
25%,NaN,NaN,NaN,NaN,0.250000
50%,NaN,NaN,NaN,NaN,0.250000
75%,NaN,NaN,NaN,NaN,0.500000


In [3]:
train_df['input'] = 'TEXT1: ' + train_df.context + '; TEXT2: ' + train_df.target + '; ANC1: ' + train_df.anchor

In [4]:

train_df['input'].head()

0    TEXT1: A47; TEXT2: abatement of pollution; ANC...
1    TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2    TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3    TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4    TEXT1: A47; TEXT2: forest region; ANC1: abatement
Name: input, dtype: object

In [5]:
from datasets import Dataset,DatasetDict

ds = Dataset.from_pandas(train_df)

d:\VS_Code_Projects\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [7]:
model_nm = 'microsoft/deberta-v3-small'

In [8]:
import os
os.environ["TRANSFORMERS_NO_TF_WARNING"] = "1"
os.environ["USE_TF"] = "0"
os.environ["USE_TORCH"] = "1"

from transformers import TrainingArguments
#print("✅ Transformers loaded with PyTorch backend only!")


In [9]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)


d:\VS_Code_Projects\.venv\Lib\site-packages\transformers\convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [10]:
#from transformers import DebertaV2Tokenizer, DebertaV2Model

# Load tokenizer
#tokenizer = DebertaV2Tokenizer.from_pretrained("microsoft/deberta-v3-small")


In [47]:
tokz.batch_encode_plus(["caaaat", "kitten", "dog"])


{'input_ids': [[1, 2285, 54906, 297, 2], [1, 22132, 2], [1, 1560, 2]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0], [0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1], [1, 1, 1]]}

In [12]:
def tok_func(x): return tokz(x["input"])

In [13]:
tok_ds = ds.map(tok_func, batched=True)

Map: 100%|██████████| 36473/36473 [00:01<00:00, 18827.23 examples/s]


In [14]:
tok_ds = tok_ds.rename_columns({'score':'labels'})

In [54]:
tok_ds.to_pandas().head()

,id,anchor,target,context,labels,input,input_ids,token_type_ids,attention_mask
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,TEXT1: A47; TEXT2: abatement of pollution; ANC...,"[1, 54453, 435, 294, 336, 5753, 346, 54453, 44...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,TEXT1: A47; TEXT2: act of abating; ANC1: abate...,"[1, 54453, 435, 294, 336, 5753, 346, 54453, 44...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,TEXT1: A47; TEXT2: active catalyst; ANC1: abat...,"[1, 54453, 435, 294, 336, 5753, 346, 54453, 44...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,TEXT1: A47; TEXT2: eliminating process; ANC1: ...,"[1, 54453, 435, 294, 336, 5753, 346, 54453, 44...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,TEXT1: A47; TEXT2: forest region; ANC1: abatement,"[1, 54453, 435, 294, 336, 5753, 346, 54453, 44...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [15]:
eval_df = pd.read_csv('data/test.csv')
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,hybrid bearing,inorganic photoconductor drum,G02
freq,1,2,1,3


In [16]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [17]:
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor

eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

Map: 100%|██████████| 36/36 [00:00<00:00, 4351.44 examples/s]


In [18]:
from transformers import TrainingArguments,Trainer

In [25]:
from transformers import TrainingArguments,Trainer
import pandas as pd
bs = 128
epochs = 4
lr = 8e-5
def corr(x,y): return np.corrcoef(x,y)[0][1]
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    eval_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')


In [26]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\diaa_\AppData\Local\Temp\ipykernel_11700\3597993663.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],


In [27]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.


Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.028845,0.796937
2,No log,0.021589,0.826480
3,0.030900,0.021432,0.834651
4,0.030900,0.022348,0.834418


d:\VS_Code_Projects\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
d:\VS_Code_Projects\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=856, training_loss=0.023604689357436707, metrics={'train_runtime': 15317.3396, 'train_samples_per_second': 7.143, 'train_steps_per_second': 0.056, 'total_flos': 715555561923540.0, 'train_loss': 0.023604689357436707, 'epoch': 4.0})

In [28]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

array([[ 0.51078546],
       [ 0.67825091],
       [ 0.58060819],
       [ 0.31784663],
       [-0.03675404],
       [ 0.40562624],
       [ 0.5134905 ],
       [-0.0087507 ],
       [ 0.36802411],
       [ 1.1097126 ],
       [ 0.20448072],
       [ 0.25217757],
       [ 0.73939878],
       [ 0.79087883],
       [ 0.78960258],
       [ 0.48605248],
       [ 0.28689802],
       [-0.03981244],
       [ 0.63429856],
       [ 0.33049425],
       [ 0.55986172],
       [ 0.23382665],
       [ 0.05135921],
       [ 0.20580459],
       [ 0.56728005],
       [-0.03149875],
       [-0.04136033],
       [-0.02480044],
       [-0.0493671 ],
       [ 0.59129459],
       [ 0.35603294],
       [-0.0176028 ],
       [ 0.70531684],
       [ 0.51339221],
       [ 0.43057898],
       [ 0.24794574]])

In [29]:
preds = np.clip(preds, 0, 1)